In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error
import tqdm
import sys
import os
import gc
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 2）探索性数据分析（EDA）
# 在数据准备阶段，主要读取训练数据和测试数据，并进行基本的数据展示。
# 加载数据
train = pd.read_csv('data/dataset/train.csv')
print("Train data is:")
print(train.head())
test = pd.read_csv('data/dataset/test.csv')
print("Test data is:")
print(test.head())

Train data is:
           id  dt  type  target
0  00037f39cf  11     2  44.050
1  00037f39cf  12     2  50.672
2  00037f39cf  13     2  39.042
3  00037f39cf  14     2  35.900
4  00037f39cf  15     2  53.888
Test data is:
           id  dt  type
0  00037f39cf   1     2
1  00037f39cf   2     2
2  00037f39cf   3     2
3  00037f39cf   4     2
4  00037f39cf   5     2


In [3]:
# 合并训练数据和测试数据
# 首先，将训练数据 (train) 和测试数据 (test) 合并成一个整体数据框 data——方便排序和平移
data = pd.concat([train, test], axis=0).reset_index(drop=True)
# 然后根据 id 和 dt（时间）进行降序排序，确保每个 id 的数据按时间从近到远排列。 ascending=Fasle 倒序
data = data.sort_values(['id','dt'], ascending=False).reset_index(drop=True)

In [4]:
# 下面正式进行特征工程的步骤

# 历史平移
# 这里进行了目标值 (target) 的平移操作，生成 target_shift{i} 特征。
# 通过 groupby 和 shift 操作，对每个 id，将目标值向后平移 i 步（从 10 到 35）。
# 例如，target_shift10 表示该行目标值是10步之前的目标值——前面按照倒序排过了
for i in range(10,36):
    data[f'target_shift{i}'] = data.groupby('id')['target'].shift(i)

# 历史平移 + 差分特征
# 对于 target_shift10 特征，生成其差分特征。
# 通过 diff 操作，计算平移后的目标值在不同步数下的差异。
# 例如，target_shift10_diff1 表示 target_shift10 和前一个时间步的差值。
for i in range(1,4):
    data[f'target_shift10_diff{i}'] = data.groupby('id')['target_shift10'].diff(i)
    
# 窗口统计
# 对目标值进行滑动窗口统计，生成均值、最大值、最小值和标准差特征。
# 使用 rolling 函数，对每个 id 进行滑动窗口操作，窗口大小分别为 15、30、50 和 70。
# min_periods=3 表示窗口内至少需要3个非缺失值才能计算统计量。
for win in [15,30,50,70]:
    data[f'target_win{win}_mean'] = data.groupby('id')['target'].rolling(window=win, min_periods=3, closed='left').mean().values
    data[f'target_win{win}_max'] = data.groupby('id')['target'].rolling(window=win, min_periods=3, closed='left').max().values
    data[f'target_win{win}_min'] = data.groupby('id')['target'].rolling(window=win, min_periods=3, closed='left').min().values
    data[f'target_win{win}_std'] = data.groupby('id')['target'].rolling(window=win, min_periods=3, closed='left').std().values

# 历史平移 + 窗口统计
# 对 target_shift10 特征进行滑动窗口统计，生成均值、最大值、最小值、和标准差特征。
# 使用 rolling 函数，窗口大小分别为 7、14、28、35、50 和 70。
# 这些特征结合了历史平移和平滑窗口的概念，更详细地捕捉时间序列中的变化。
for win in [7,14,28,35,50,70]:
    data[f'target_shift10_win{win}_mean'] = data.groupby('id')['target_shift10'].rolling(window=win, min_periods=3, closed='left').mean().values
    data[f'target_shift10_win{win}_max'] = data.groupby('id')['target_shift10'].rolling(window=win, min_periods=3, closed='left').max().values
    data[f'target_shift10_win{win}_min'] = data.groupby('id')['target_shift10'].rolling(window=win, min_periods=3, closed='left').min().values
    data[f'target_shift10_win{win}_sum'] = data.groupby('id')['target_shift10'].rolling(window=win, min_periods=3, closed='left').sum().values
    data[f'target_shift710win{win}_std'] = data.groupby('id')['target_shift10'].rolling(window=win, min_periods=3, closed='left').std().values

In [5]:
# 进行数据切分
# 选取了所有target值非空的记录，并重置索引，生成训练数据train。
train = data[data.target.notnull()].reset_index(drop=True)
print("train前5行数据")
print(train.head())

# 选取了所有target值为空的记录，并重置索引，生成测试数据test。
test = data[data.target.isnull()].reset_index(drop=True)
print("test前5行数据")
print(test.head())

# 确定输入特征
# 使用列表推导式选取data中的所有列名，排除id和target列，生成用于模型训练的特征列表train_cols。
train_cols = [f for f in data.columns if f not in ['id','target']]
print("列名")
print(train_cols)

train前5行数据
           id   dt  type  target  target_shift10  target_shift11  \
0  fff81139a7  506     5  18.145             NaN             NaN   
1  fff81139a7  505     5  22.021             NaN             NaN   
2  fff81139a7  504     5  21.282             NaN             NaN   
3  fff81139a7  503     5  22.818             NaN             NaN   
4  fff81139a7  502     5  28.552             NaN             NaN   

   target_shift12  target_shift13  target_shift14  target_shift15  ...  \
0             NaN             NaN             NaN             NaN  ...   
1             NaN             NaN             NaN             NaN  ...   
2             NaN             NaN             NaN             NaN  ...   
3             NaN             NaN             NaN             NaN  ...   
4             NaN             NaN             NaN             NaN  ...   

   target_shift10_win50_mean  target_shift10_win50_max  \
0                        NaN                       NaN   
1                  

In [6]:
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [7]:
from lightgbm import log_evaluation, early_stopping
# verbose_eval: 每200次迭代输出一次结果。
# early_stopping_rounds: 如果在200次迭代后模型性能没有提升，则停止训练。
callbacks = [log_evaluation(period=500), early_stopping(stopping_rounds=500)]

def cv_model(clf, train_x, train_y, test_x, clf_name, seed = 2024):
    # '''
    # clf: 传入的模型对象，可以是 LightGBM、XGBoost 或 CatBoost。
    # train_x: 训练数据特征。
    # train_y: 训练数据标签。
    # test_x: 测试数据特征。
    # clf_name: 使用模型的名称（'lgb'、'xgb' 或 'cat'）。
    # seed: 随机种子，默认值为 2024。42，56，3407，4096
    # '''

    # 使用 KFold 进行 5 折交叉验证。KFold 将数据分成 5 份，每一份轮流作为验证集，其余作为训练集。
    # 初始化 oof（out-of-fold predictions）数组，用于存储每折验证集的预测结果。
    # 初始化 test_predict 数组，用于存储每折测试集的预测结果，最后取平均。
    # 初始化 cv_scores 列表，用于存储每折的评估分数。
    folds = 5
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    oof = np.zeros(train_x.shape[0])
    test_predict = np.zeros(test_x.shape[0])
    cv_scores = []
    
    #  循环遍历每一折，分割训练集和验证集。
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        

        if clf_name == "lgb":
        # 数据准备：
        # train_matrix 和 valid_matrix 是 LightGBM 的 Dataset 对象，分别包含训练集和验证集数据及其标签。
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)
            params = {
                'boosting_type': 'gbdt',    #提升类型，使用梯度提升决策树（GBDT）。
                'objective': 'regression',  #目标函数，设为回归任务。
                'metric': 'mae',            #评价指标，使用平均绝对误差（MAE）。
                'min_child_weight': 6,      # 最小叶子节点样本权重和，用于防止过拟合。
                'num_leaves': 2 ** 6,       #叶子节点数目，设置为 64。
                'lambda_l2': 10,            #L2 正则化系数，防止过拟合。
                'feature_fraction': 0.8,    #每次分裂时选择特征的比例。
                'bagging_fraction': 0.8,    #每次迭代时数据采样的比例。
                'bagging_freq': 4,          #Bagging 的频率。
                'learning_rate': 0.1,       #学习率。
                'seed': 2023,               #随机种子。
                'nthread' : 16,             #使用的线程数。
                'verbose' : -1,             #日志级别，表示不输出任何训练日志
            }
            # 模型训练：
            # 调用 train 函数，使用指定参数进行训练，最大迭代次数为 1000。
            # valid_sets 指定验证集，用于监控训练过程中的模型性能。
            # callbacks 包含日志记录和早停回调函数。
            model = clf.train(params, 
                              train_matrix, 
                              1000, 
                              valid_sets=[train_matrix, valid_matrix],
                              categorical_feature=[], 
                              callbacks=callbacks
                              )
            
            # 模型预测：
            # predict 函数对验证集和测试集进行预测，使用最佳迭代次数（best_iteration）。
            # 在训练过程中，LightGBM 会通过监控验证集的性能来决定最优的迭代次数。
            # 这就是所谓的“早停”（early stopping）策略。
            # 如果在一定数量的迭代（stopping_rounds）之后，验证集上的性能没有提升，训练就会停止，避免模型过拟合。
            # model.best_iteration 是指在训练过程中，模型在验证集上表现最好的迭代次数。
            # 使用最佳迭代次数对验证集 val_x 进行预测。
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            # 使用最佳迭代次数对测试集 test_x 进行预测。
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
        

        if clf_name == "xgb":

        # 数据准备：
        # train_matrix、valid_matrix 和 test_matrix 是 XGBoost 的 DMatrix 对象，分别包含训练集、验证集和测试集数据及其标签。
        # DMatrix 是 XGBoost 中的数据格式，用于高效地存储和处理数据。
        # train_matrix、valid_matrix 和 test_matrix 分别是训练集、验证集和测试集的数据矩阵。
        # trn_x 和 trn_y 是当前折的训练数据的特征和标签。
        # val_x 和 val_y 是当前折的验证数据的特征和标签。
        # test_x 是测试数据的特征。
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)

        # watchlist 是一个元组列表，用于在训练过程中监控训练集和验证集的性能。
        # train_matrix 和 valid_matrix 是之前定义的 XGBoost 数据矩阵。
        # watchlist 中的元组格式为 (DMatrix, 名称)，其中名称用于标识输出日志中对应的数据集。           
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]

            xgb_params = {
              'booster': 'gbtree',              #提升器类型，使用基于树的模型。
              'objective': 'reg:squarederror',  #目标函数，设为平方误差回归。
              'eval_metric': 'mae',             #评价指标，使用平均绝对误差（MAE）。
              'max_depth': 5,                   #树的最大深度。
              'lambda': 10,                     #L2 正则化系数。
              'subsample': 0.7,                 #每次迭代时数据采样的比例。
              'colsample_bytree': 0.7,          # 每棵树使用的特征比例。
              'colsample_bylevel': 0.7,         # 每一层使用的特征比例。
              'eta': 0.1,                       #学习率。
              'tree_method': 'hist',            #树的构建方法，使用直方图优化（hist）。
              'seed': 520,                      #随机种子。
              'nthread': 16                     # 使用的线程数。
              }
            
            # 模型训练：
            # 调用 train 函数，使用指定参数进行训练，最大迭代次数为 1000。
            # evals 指定验证集，用于监控训练过程中的模型性能。
            # verbose_eval 每 200 次迭代输出一次结果。
            # early_stopping_rounds 指定早停轮数，如果在 100 次迭代后性能没有提升，则停止训练。
            model = clf.train(xgb_params, 
                              train_matrix, 
                              num_boost_round=1000, 
                              evals=watchlist, 
                              verbose_eval=200, 
                              early_stopping_rounds=100
                              )
            
            val_pred  = model.predict(valid_matrix)
            test_pred = model.predict(test_matrix)
        


        if clf_name == "cat":
            params = {'learning_rate': 0.1,         #学习率。
                      'depth': 5,                   #树的深度。
                      'bootstrap_type':'Bernoulli', #采样类型，使用 Bernoulli 采样。
                      'random_seed':2023,           #随机种子。
                      'od_type': 'Iter',            #过拟合检测类型，使用迭代类型。
                      'od_wait': 100,               #过拟合检测等待的轮数。
                      'random_seed': 11, 
                      'allow_writing_files': False  #是否允许写入文件，设为 False 防止产生不必要的文件。
                      }
        
            # 模型训练：
            # 创建 CatBoost 模型对象，指定最大迭代次数为 1000。
            # 调用 fit 函数，进行模型训练，设置验证集用于监控模型性能。
            # metric_period 每 200 次迭代输出一次结果。
            # use_best_model 启用选择最佳模型。
            # cat_features 指定分类特征列表，这里为空。
            # verbose 控制输出日志的详细程度。
            model = clf(iterations=1000, **params)
            model.fit(trn_x, trn_y, 
                      eval_set=(val_x, val_y),
                      metric_period=200,
                      use_best_model=True, 
                      cat_features=[],
                      verbose=1)
            
            # 模型预测：
            # predict 函数对验证集和测试集进行预测。
            val_pred  = model.predict(val_x)
            test_pred = model.predict(test_x)
        
        # 将验证集预测结果存储到 oof 中。
        # 累加每折的测试集预测结果。
        # 计算验证集的平均绝对误差（MAE），并存储到 cv_scores 中。
        oof[valid_index] = val_pred
        test_predict += test_pred / kf.n_splits
        
        score = mean_absolute_error(val_y, val_pred)
        cv_scores.append(score)
        print(cv_scores)
    #返回 out-of-fold 预测结果和测试集预测结果。
    return oof, test_predict

# 选择lightgbm模型
lgb_oof, lgb_test = cv_model(lgb, train[train_cols], train['target'], test[train_cols], 'lgb')
# 选择xgboost模型
xgb_oof, xgb_test = cv_model(xgb, train[train_cols], train['target'], test[train_cols], 'xgb')
# 选择catboost模型
cat_oof, cat_test = cv_model(CatBoostRegressor, train[train_cols], train['target'], test[train_cols], 'cat')

# 进行取平均融合
final_test = (lgb_test + xgb_test + cat_test) / 3

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np

In [9]:
# 函数参数说明
# clf: 传入的模型对象，可以是 LightGBM、XGBoost 或 CatBoost。
# train_x: 训练数据的特征。
# train_y: 训练数据的标签。
# test_x: 测试数据的特征。
# clf_name: 使用模型的名称（'lgb'、'xgb' 或 'cat'）。
# seed: 随机种子，默认值为 2024。
# param_search: 参数搜索的范围，如果需要进行超参数优化，可以传入相应的参数范围。default=None
# search_type: 搜索类型，默认为 'grid'（网格搜索）。
def cv_model(clf, train_x, train_y, test_x, clf_name, seed=2024, param_search=None, search_type='grid'):

# KFold交叉验证
# 定义 5 折交叉验证，使用 KFold 将数据分成 5 份，每一份轮流作为验证集，其余作为训练集。
# shuffle=True 确保数据在分割之前被随机打乱，random_state=seed 用于保证结果可复现。
# 初始化 oof（out-of-fold predictions）数组，用于存储每折验证集的预测结果。
# 初始化 test_predict 数组，用于存储每折测试集的预测结果，最后取平均。
# 初始化 cv_scores 列表，用于存储每折的评估分数。
    folds = 5
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    oof = np.zeros(train_x.shape[0])
    test_predict = np.zeros(test_x.shape[0])
    cv_scores = []
    
# 循环遍历每一折的训练和验证集索引，通过 kf.split(train_x, train_y) 获取当前折的训练集和验证集索引。
# 使用 train_index 和 valid_index 分割训练数据和验证数据。
# trn_x 和 trn_y 分别是当前折的训练集特征和标签。
# val_x 和 val_y 分别是当前折的验证集特征和标签。  
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        # 打印当前折的编号，方便跟踪训练过程。  
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        
        if clf_name == "lgb":

            params = {
                'boosting_type': 'gbdt',    #提升类型，使用梯度提升决策树（GBDT）。
                'objective': 'regression',  #目标函数，设为回归任务。
                'metric': 'mae',            #评价指标，使用平均绝对误差（MAE）。
                'min_child_weight': 6,      # 最小叶子节点样本权重和，用于防止过拟合。
                'num_leaves': 2 ** 6,       #叶子节点数目，设置为 64。
                'lambda_l2': 10,            #L2 正则化系数，防止过拟合。
                'feature_fraction': 0.8,    #每次分裂时选择特征的比例。
                'bagging_fraction': 0.8,    #每次迭代时数据采样的比例。
                'bagging_freq': 4,          #Bagging 的频率。
                'learning_rate': 0.1,       #学习率。
                'seed': 42,               #随机种子。
                'nthread' : 16,             #使用的线程数。
                'verbose' : -1,             #日志级别，表示不输出任何训练日志
            }

            # 看是否传入参数parm_search(!=None)
            if param_search:

                # 如果 search_type 为 'grid'，则使用 GridSearchCV 进行网格搜索。
                # clf.LGBMRegressor(**params)：创建一个 LightGBM 回归模型，使用传入的 params 参数初始化。
                # param_search：定义了需要搜索的参数范围。
                # scoring='neg_mean_absolute_error'：使用负平均绝对误差（MAE）作为评分标准。
                # cv=3：使用 3 折交叉验证。
                # # verbose=1：输出详细日志信息。
                if search_type == 'grid':
                    search = GridSearchCV(estimator=lgb.LGBMRegressor(**params), param_grid=param_search, scoring='neg_mean_absolute_error', cv=3, verbose=1)
                    
                # 如果 search_type 为 'random'，则使用 RandomizedSearchCV 进行随机搜索。
                # n_iter=50：指定随机搜索的迭代次数为 50。
                # random_state=seed：设置随机种子，确保结果可复现。                
                elif search_type == 'random':
                    search = RandomizedSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, verbose=1, random_state=seed)
                    
                # 如果 search_type 为其他值，默认使用 BayesSearchCV 进行贝叶斯优化搜索。
                # n_iter=50：指定搜索的迭代次数为 50。                
                else:
                    search = BayesSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, random_state=seed)
                    
                # 使用搜索得到的最佳参数更新 params。
                # search.best_params_ 包含了搜索过程中找到的最优参数。
                search.fit(trn_x, trn_y)
                params.update(search.best_params_)
            
            #train_matrix 和 valid_matrix 是 LightGBM 的 Dataset 对象，分别包含训练集和验证集数据及其标签。
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            # 模型训练：
            # 调用 train 函数，使用指定参数进行训练，最大迭代次数为 1000。
            # valid_sets 指定验证集，用于监控训练过程中的模型性能。
            # callbacks 包含日志记录和早停回调函数。
            model = clf.train(params, 
                              train_matrix, 
                              1000, 
                              valid_sets=[train_matrix, valid_matrix],
                              categorical_feature=[], 
                              callbacks=callbacks
                              )
            
            # 模型预测：
            # predict 函数对验证集和测试集进行预测，使用最佳迭代次数（best_iteration）。
            # 在训练过程中，LightGBM 会通过监控验证集的性能来决定最优的迭代次数。
            # 这就是所谓的“早停”（early stopping）策略。
            # 如果在一定数量的迭代（stopping_rounds）之后，验证集上的性能没有提升，训练就会停止，避免模型过拟合。
            # model.best_iteration 是指在训练过程中，模型在验证集上表现最好的迭代次数。
            # 使用最佳迭代次数对验证集 val_x 进行预测。
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            # 使用最佳迭代次数对测试集 test_x 进行预测。
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
        
        if clf_name == "xgb":
            
            xgb_params = {
              'booster': 'gbtree',              #提升器类型，使用基于树的模型。
              'objective': 'reg:squarederror',  #目标函数，设为平方误差回归。
              'eval_metric': 'mae',             #评价指标，使用平均绝对误差（MAE）。
              'max_depth': 5,                   #树的最大深度。
              'lambda': 10,                     #L2 正则化系数。
              'subsample': 0.7,                 #每次迭代时数据采样的比例。
              'colsample_bytree': 0.7,          # 每棵树使用的特征比例。
              'colsample_bylevel': 0.7,         # 每一层使用的特征比例。
              'eta': 0.1,                       #学习率。
              'tree_method': 'hist',            #树的构建方法，使用直方图优化（hist）。
              'seed': 42,                      #随机种子。
              'nthread': 16                     # 使用的线程数。
              }
            
            if param_search:

                # 如果 search_type 为 'grid'，则使用 GridSearchCV 进行网格搜索。
                # clf.LGBMRegressor(**params)：创建一个 LightGBM 回归模型，使用传入的 params 参数初始化。
                # param_search：定义了需要搜索的参数范围。
                # scoring='neg_mean_absolute_error'：使用负平均绝对误差（MAE）作为评分标准。
                # cv=3：使用 3 折交叉验证。
                # # verbose=1：输出详细日志信息。
                if search_type == 'grid':
                    search = GridSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, verbose=1)
                    
                # 如果 search_type 为 'random'，则使用 RandomizedSearchCV 进行随机搜索。
                # n_iter=50：指定随机搜索的迭代次数为 50。
                # random_state=seed：设置随机种子，确保结果可复现。                
                elif search_type == 'random':
                    search = RandomizedSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, verbose=1, random_state=seed)
                    
                # 如果 search_type 为其他值，默认使用 BayesSearchCV 进行贝叶斯优化搜索。
                # n_iter=50：指定搜索的迭代次数为 50。                
                else:
                    search = BayesSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, random_state=seed)
                    
                # 使用搜索得到的最佳参数更新 params。
                # search.best_params_ 包含了搜索过程中找到的最优参数。
                search.fit(trn_x, trn_y)
                params.update(search.best_params_)
            
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(xgb_params, train_matrix, num_boost_round=1000, evals=watchlist, verbose_eval=200, early_stopping_rounds=100)
            val_pred  = model.predict(valid_matrix)
            test_pred = model.predict(test_matrix)
            
        if clf_name == "cat":
            params = {'learning_rate': 0.1, 'depth': 5, 'bootstrap_type':'Bernoulli','random_seed':42,
                      'od_type': 'Iter', 'od_wait': 100, 'random_seed': 11, 'allow_writing_files': False}

            if param_search:

                # 如果 search_type 为 'grid'，则使用 GridSearchCV 进行网格搜索。
                # clf.LGBMRegressor(**params)：创建一个 LightGBM 回归模型，使用传入的 params 参数初始化。
                # param_search：定义了需要搜索的参数范围。
                # scoring='neg_mean_absolute_error'：使用负平均绝对误差（MAE）作为评分标准。
                # cv=3：使用 3 折交叉验证。
                # # verbose=1：输出详细日志信息。
                if search_type == 'grid':
                    search = GridSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, verbose=1)
                    
                # 如果 search_type 为 'random'，则使用 RandomizedSearchCV 进行随机搜索。
                # n_iter=50：指定随机搜索的迭代次数为 50。
                # random_state=seed：设置随机种子，确保结果可复现。                
                elif search_type == 'random':
                    search = RandomizedSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, verbose=1, random_state=seed)
                    
                # 如果 search_type 为其他值，默认使用 BayesSearchCV 进行贝叶斯优化搜索。
                # n_iter=50：指定搜索的迭代次数为 50。                
                else:
                    search = BayesSearchCV(lgb.LGBMRegressor(**params), param_search, scoring='neg_mean_absolute_error', cv=3, n_iter=50, random_state=seed)
                    
                # 使用搜索得到的最佳参数更新 params。
                # search.best_params_ 包含了搜索过程中找到的最优参数。
                search.fit(trn_x, trn_y)
                params.update(search.best_params_)

            model = clf(iterations=1000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      metric_period=200,
                      use_best_model=True, 
                      verbose=1)
            
            val_pred  = model.predict(val_x)
            test_pred = model.predict(test_x)
        
        oof[valid_index] = val_pred
        test_predict += test_pred / kf.n_splits
        
        score = mean_absolute_error(val_y, val_pred)
        cv_scores.append(score)
        print(cv_scores)
        
    return oof, test_predict

# 参数搜索空间示例
param_search = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500]
}

# 选择lightgbm模型并进行参数优化
lgb_oof, lgb_test = cv_model(lgb, train[train_cols], train['target'], test[train_cols], 'lgb', param_search, search_type='grid')

# 参数搜索空间示例
param_search = {
    'max_depth': [4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500]
}

# 选择xgboost模型并进行参数优化
xgb_oof, xgb_test = cv_model(xgb, train[train_cols], train['target'], test[train_cols], 'xgb', param_search, search_type='random')

# 参数搜索空间示例
param_search = {
    'depth': [4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1]
}

# 选择catboost模型并进行参数优化
cat_oof, cat_test = cv_model(CatBoostRegressor, train[train_cols], train['target'], test[train_cols], 'cat', param_search, search_type='bayes')

# 进行取平均融合
final_test1 = (lgb_test + xgb_test + cat_test) / 3


************************************ 1 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 6.55896	valid_1's l1: 6.82078
[1000]	training's l1: 6.31344	valid_1's l1: 6.72825
Did not meet early stopping. Best iteration is:
[998]	training's l1: 6.31392	valid_1's l1: 6.72819
[6.7281933623999235]
************************************ 2 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 6.56275	valid_1's l1: 6.81872
[1000]	training's l1: 6.31979	valid_1's l1: 6.72998
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 6.31979	valid_1's l1: 6.72998
[6.7281933623999235, 6.729978403365649]
************************************ 3 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 6.5647	valid_1's l1: 6.80756
[1000]	training's l1: 6.31943	valid_1's l1: 6.71652
Did not meet early

0:	learn: 46.3013554	test: 46.4794383	best: 46.4794383 (0)	total: 352ms	remaining: 5m 52s
200:	learn: 14.0004515	test: 14.0463593	best: 14.0463593 (200)	total: 41.6s	remaining: 2m 45s
400:	learn: 13.4367973	test: 13.6020852	best: 13.6020852 (400)	total: 1m 24s	remaining: 2m 5s
600:	learn: 13.1351201	test: 13.3900200	best: 13.3900200 (600)	total: 2m 6s	remaining: 1m 23s
800:	learn: 12.9029584	test: 13.2496119	best: 13.2496119 (800)	total: 2m 51s	remaining: 42.7s
999:	learn: 12.7313688	test: 13.1598318	best: 13.1598318 (999)	total: 3m 34s	remaining: 0us

bestTest = 13.15983176
bestIteration = 999

[7.064867606955804]
************************************ 2 ************************************


0:	learn: 46.3653360	test: 46.1890329	best: 46.1890329 (0)	total: 204ms	remaining: 3m 24s
200:	learn: 13.9958084	test: 14.0944545	best: 14.0944545 (200)	total: 41.9s	remaining: 2m 46s
400:	learn: 13.4364200	test: 13.6327754	best: 13.6327754 (400)	total: 1m 24s	remaining: 2m 6s
600:	learn: 13.1105704	test: 13.4130411	best: 13.4129390 (597)	total: 2m 7s	remaining: 1m 24s
800:	learn: 12.8913936	test: 13.2810127	best: 13.2810127 (800)	total: 2m 47s	remaining: 41.6s
999:	learn: 12.7170494	test: 13.1842582	best: 13.1842582 (999)	total: 3m 27s	remaining: 0us

bestTest = 13.18425819
bestIteration = 999

[7.064867606955804, 7.064302682173274]
************************************ 3 ************************************


0:	learn: 46.3143483	test: 46.4145671	best: 46.4145671 (0)	total: 206ms	remaining: 3m 25s
200:	learn: 14.0304791	test: 14.0064205	best: 14.0064205 (200)	total: 43.9s	remaining: 2m 54s
400:	learn: 13.4455836	test: 13.5200526	best: 13.5200526 (400)	total: 1m 28s	remaining: 2m 11s
600:	learn: 13.1652347	test: 13.3297732	best: 13.3297732 (600)	total: 2m 8s	remaining: 1m 25s
800:	learn: 12.9478597	test: 13.1898428	best: 13.1898428 (800)	total: 2m 46s	remaining: 41.4s
999:	learn: 12.7713201	test: 13.0889703	best: 13.0889703 (999)	total: 3m 25s	remaining: 0us

bestTest = 13.08897033
bestIteration = 999

[7.064867606955804, 7.064302682173274, 7.055804377966254]
************************************ 4 ************************************


0:	learn: 46.3708450	test: 46.1735689	best: 46.1735689 (0)	total: 208ms	remaining: 3m 28s
200:	learn: 14.0154822	test: 14.0881818	best: 14.0881818 (200)	total: 38.9s	remaining: 2m 34s
400:	learn: 13.4149642	test: 13.6015942	best: 13.6015942 (400)	total: 1m 17s	remaining: 1m 55s
600:	learn: 13.1134896	test: 13.3881497	best: 13.3881497 (600)	total: 1m 57s	remaining: 1m 17s
800:	learn: 12.9126392	test: 13.2645631	best: 13.2645631 (800)	total: 2m 35s	remaining: 38.7s
999:	learn: 12.7359276	test: 13.1501330	best: 13.1501330 (999)	total: 3m 14s	remaining: 0us

bestTest = 13.15013298
bestIteration = 999

[7.064867606955804, 7.064302682173274, 7.055804377966254, 7.065730191652328]
************************************ 5 ************************************


0:	learn: 46.3081884	test: 46.4182915	best: 46.4182915 (0)	total: 183ms	remaining: 3m 2s
200:	learn: 13.8979556	test: 14.2178675	best: 14.2178675 (200)	total: 38.5s	remaining: 2m 33s
400:	learn: 13.3531667	test: 13.7593956	best: 13.7593956 (400)	total: 1m 17s	remaining: 1m 55s
600:	learn: 13.0578200	test: 13.5456391	best: 13.5456391 (600)	total: 1m 56s	remaining: 1m 17s
800:	learn: 12.8526857	test: 13.4182427	best: 13.4182427 (800)	total: 2m 35s	remaining: 38.7s
999:	learn: 12.6784698	test: 13.3173631	best: 13.3173631 (999)	total: 3m 14s	remaining: 0us

bestTest = 13.31736306
bestIteration = 999

[7.064867606955804, 7.064302682173274, 7.055804377966254, 7.065730191652328, 7.063521892130133]


In [10]:
# 保存结果文件到本地
test['target'] = final_test1
test[['id','dt','target']].to_csv('submit.csv', index=None)

In [11]:
from sklearn.linear_model import Ridge

In [12]:
#stacking融合
# 参数：
# oof_1, oof_2, oof_3: 模型1、模型2、模型3的 out-of-fold (OOF) 预测结果，
# 分别对应 LightGBM、XGBoost 和 CatBoost 模型的 OOF 预测。
# predictions_1, predictions_2, predictions_3: 模型1、模型2、模型3对测试集的预测结果，
# 分别对应 LightGBM、XGBoost 和 CatBoost 模型的测试集预测。
# y: 训练数据的实际标签。
def stack_model(oof_1, oof_2, oof_3, predictions_1, predictions_2, predictions_3, y):

    # '''
    # 输入的oof_1, oof_2, oof_3可以对应lgb_oof，xgb_oof，cat_oof
    # predictions_1, predictions_2, predictions_3对应lgb_test，xgb_test，cat_test
    # '''

    # 1.创建训练和测试数据的堆叠：
    # 将 oof_1, oof_2, oof_3 拼接成训练数据的堆叠矩阵 train_stack。
    train_stack = pd.concat([oof_1, oof_2, oof_3], axis=1)
    # 将 predictions_1, predictions_2, predictions_3 拼接成测试数据的堆叠矩阵 test_stack。
    test_stack = pd.concat([predictions_1, predictions_2, predictions_3], axis=1)
    
    # 2.初始化存储变量：
    # oof: 存储堆叠模型的 OOF 预测结果。shape[0]返回行数
    # predictions: 存储堆叠模型的测试集预测结果。
    # scores: 存储每一折交叉验证的 MAE 分数。
    oof = np.zeros((train_stack.shape[0],))
    predictions = np.zeros((test_stack.shape[0],))
    scores = []
    
    # 3.定义交叉验证：
    # 使用 RepeatedKFold 进行交叉验证，设置 n_splits=5 和 n_repeats=2，即5折交叉验证重复2次。
    from sklearn.model_selection import RepeatedKFold
    folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
    
    # 4.交叉验证训练和预测：
    # 循环遍历每一折交叉验证，将数据分为训练集和验证集。
    # 初始化一个 Ridge 回归模型进行训练。
    # 使用训练好的模型对验证集和测试集进行预测。
    # 计算每一折的 MAE 分数并存储。
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, train_stack)): 
        print("fold n°{}".format(fold_+1))
        trn_data, trn_y = train_stack.loc[trn_idx], y[trn_idx]
        val_data, val_y = train_stack.loc[val_idx], y[val_idx]
        
        clf = Ridge(random_state=42)
        clf.fit(trn_data, trn_y)

        oof[val_idx] = clf.predict(val_data)
        predictions += clf.predict(test_stack) / (5 * 2)
        
        score_single = mean_absolute_error(val_y, oof[val_idx])
        scores.append(score_single)
        print(f'{fold_+1}/{5}', score_single)
    print('mean: ',np.mean(scores))
   
    # 5.返回结果：
    # oof: 堆叠模型的 OOF 预测结果。
    # predictions: 堆叠模型的测试集预测结果。
    return oof, predictions

# 6.调用 stack_model 函数：
# 传入 LightGBM、XGBoost 和 CatBoost 模型的 OOF 预测和测试集预测，以及训练数据的标签 train['target']。
# 获取堆叠模型的 OOF 预测结果 stack_oof 和测试集预测结果 stack_pred。
stack_oof, stack_pred = stack_model(pd.DataFrame(lgb_oof), pd.DataFrame(xgb_oof), pd.DataFrame(cat_oof), 
                                    pd.DataFrame(lgb_test), pd.DataFrame(xgb_test), pd.DataFrame(cat_test), train['target'])
# 7.输出
# 将堆叠模型的测试集预测结果 stack_pred 赋值给 final_test。
final_test=stack_pred

fold n°1
1/5 6.725996253671618
fold n°2
2/5 6.726375409966235
fold n°3
3/5 6.71257118891519
fold n°4
4/5 6.71954283665174
fold n°5
5/5 6.7293882506098965
fold n°6
6/5 6.71210481424597
fold n°7
7/5 6.7221608755847235
fold n°8
8/5 6.714667900893997
fold n°9
9/5 6.733736214937944
fold n°10
10/5 6.73100587455158
mean:  6.722754962002888


In [13]:
# 保存结果文件到本地
test['target'] = final_test
test[['id','dt','target']].to_csv('submit3.csv', index=None)